In [ ]:
#seqkitなど諸々コマンドのインストール

conda activate RNAseq
mamba install -c bioconda minimap2 samtools bowtie2 igv seqkit 

In [ ]:
#ファイルの中身を確認する

In [ ]:
seqkit stats "file"　fileはgz形式

In [ ]:
#中身を1つのメモかエクセルにまとめる

In [ ]:
seqkit stats -T "*_001.fastq.gz" > "statistics.tsv"

In [ ]:
　> "file.tsv"で保存
　tsvの拡張子をつけると、エクセルのファイルになる
　-Tはタブ区切りにするオプション

In [ ]:
　普通に開くとメモ帳になるので、エクセルファイル→開く→参照→すべてのファイルにして作ったファイルを開く

In [ ]:
#gzファイルの解凍

In [ ]:
gzip -dc "WT-2_R1_001.fastq.gz"  > "R1.fastq"

In [ ]:
#リードの前処理（クリーニング）

In [ ]:
#環境のアクティブ化 
conda activate RNAseq 
#=>プロンプトが(RNAseq)に変わったことを確認
#必要なソフトのインストール
mamba install -c bioconda fastp fastqc seqkit

In [ ]:
#"ATTCCATA"を検索　grep　
　ただし配列名に入っている"ATTCCATA"は除く（配列名が@から始まるので、-vのオプションで@から始まるものは除いている）

In [ ]:
grep -v "@" "R1.fastq" |grep "ATTCCATA"

In [ ]:
　"R1.fastq"は上で解凍したファイル

In [ ]:
#行数を数える wc -l

In [ ]:
grep -v "@" R1.fastq |grep "ATTCCATA"|wc -l

In [ ]:
　全リード数÷ここで出てきた行数が、全リード中のアダプターの割合

In [ ]:
#fastpのインストール

In [ ]:
mamba install -c bioconda fastp 

In [ ]:
#fastpの実行
　最後の数字を変えると、クオリティなどが変わる

In [ ]:
fastp -i "WT-2_R1_001.fastq.gz" -I "WT-2_R2_001.fastq.gz" -3 -o "out_pair1.fq.gz" -O "out_pair2.fq.gz" \-h "report.html" -j "report.json" -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1

In [ ]:
　"WT-2_R1_001.fastq.gz"と"WT-2_R2_001.fastq.gz"はペアのリード
　"out_pair1.fq.gz"と"out_pair2.fq.gz"はそれぞれの出力ファイル名
 　出力された"report.html"を押すと、chromeでfastp reportが見れる

In [ ]:
#さらにクリーニングする？並列して実行？

In [ ]:
fastp -i "WT-2_R1_001.fastq.gz" -I "WT-2_R2_001.fastq.gz" -3 -o "out_pair1.fq.gz" -O "out_pair2.fq.gz" \-h "report.html" -j "report.json" -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1 --adapter_sequence "ATTCCATA" --adapter_sequence_r2 "CGCTCCAC"

In [ ]:
　--adapter_sequenceの後の　"配列"はSEQ_REPORTのアダプター配列に変える

In [ ]:
#gzファイルの解凍

In [ ]:
gzip -dc "out_pair1.fq.gz"  > "out_pair1.fq"

In [ ]:
grep -v "@" out_pair1.fq |grep "ATTCCATA"|wc -l

In [ ]:
fastp -i "dg237_R1_001.fastq.gz" -I "dg237_R2_001.fastq.gz" -3 -o "out_pair3.fq.gz" -O "out_pair4.fq.gz" \-h "report2.html" -j "report2.json" -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1

In [ ]:
fastp -i "dg237_R1_001.fastq.gz" -I "dg237_R2_001.fastq.gz" -3 -o "out_pair3.fq.gz" -O "out_pair4.fq.gz" \-h "report2.html" -j "report2.json" -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1 --adapter_sequence "AGGTCAGA" --adapter_sequence_r2 "CGCTCCAC"

In [ ]:
fastp -i "BR10PR22_R1_001.fastq.gz" -I "BR10PR22_R2_001.fastq.gz" -3 -o "out_pair5.fq.gz" -O "out_pair6.fq.gz" \-h "report3.html" -j "report3.json" -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1

In [ ]:
fastp -i "BR10PR22_R1_001.fastq.gz" -I "BR10PR22_R2_001.fastq.gz" -3 -o "out_pair5.fq.gz" -O "out_pair6.fq.gz" \-h "report3.html" -j "report3.json" -q 20 -t 1 -T 1 -l 20 -w 8 -f 1 -F 1--adapter_sequence "GACGAGAT" --adapter_sequence_r2 "ATTAACAA"

In [ ]:
#これで全データ綺麗になった

In [ ]:
#6データを１つにまとめる

In [ ]:
seqkit stats -T "out_pair*.fq.gz" > "statistics2.tsv"

In [ ]:
　これをエクセルで表にまとめる

In [ ]:
#マッピング bowtie2のインストール

In [ ]:
mamba install -c bioconda Bowtie2

In [ ]:
#indexing

In [ ]:
bowtie2-build "GCA_002142475.1.fna" 'bowite2_index'

In [ ]:
"GCA_002142475.1.fna"はleptolyngbyaの全ゲノム

In [ ]:
#mapping

In [ ]:
bowtie2 --sensitive -x 'bowite2_index' -1 "out_pair1.fq.gz" -2 "out_pair2.fq.gz"|samtools sort -@ 4 - > out.bam

In [ ]:
(#重たいとき
bowtie2 -x 'bowite2_index' -1 "out_pair1.fq.gz" -2 "out_pair2.fq.gz" > "out.sam"
samtools sort -@ 4 "out.sam" > "out.bam")

In [ ]:
samtools index -@ 4 "out.bam"

In [ ]:
samtools flagstats "out.bam"

In [ ]:
#igvの起動

In [ ]:
igv -g GCA_002142475.1.fna out.bam,out2.bam,out3.bam

In [ ]:
　ファイルは、カンマで区切る

In [ ]:
#breseq

In [ ]:
breseq -r Leptolyngbya_boryana_IAM101.gbk -o WT.gbk out_pair1.fq.gz out_pair2.fq.gz

In [ ]:
 gbkがreference

In [ ]:
#breseqのgdファイルを1つのファイルにまとめる

In [ ]:
gdtools COMPARE -o integrate.html -r Leptolyngbya_boryana_IAM101.gbk WT-2.gd BR10PR22.gd dg237.gd